# `polaris adme`

This notebook demonstrates using the Mordred(-community) molecular descriptors with Neural Pairwise Regression (via `nepare`) with the `polaris` benchmarking library.

## Requirements
Python 3.10+ (originally run on 3.12)
 - polaris-lib
 - pandas
 - fastprop
 - mordredcommunity
 - rdkit
 - lightning
 - torch
 - numpy
 - ipywidgets

You will also need to run `pip install .` in the repository's root directory to install `nepare`.

## `polaris` Setup

After running `polaris login` on the command line, we can import everything (checking that the version is recent enough) and then download the benchmark data.

In [1]:
import polaris as po
import pandas as pd

In [2]:
from packaging.version import Version
assert Version(po.__version__) >= Version("0.11.6"), "test.as_dataframe does not work in earlier versions of Polaris, please upgrade"

`polaris` makes it really easy to run different benchmarks quickly - just change the name inside `load_benchmark` to try something else.
I'm using this same notebook for a few different benchmarks, all from the Fang biogen ADME paper (https://pubs.acs.org/doi/abs/10.1021/acs.jcim.3c00160) which have been made conveniently available on `polaris`.

In [3]:
%%capture
# https://polarishub.io/benchmarks/polaris/adme-fang-rppb-1
benchmark = po.load_benchmark("polaris/adme-fang-RPPB-1")
# https://polarishub.io/benchmarks/polaris/adme-fang-solu-1
# benchmark = po.load_benchmark("polaris/adme-fang-SOLU-1")

In [4]:
train, test = benchmark.get_train_test_split()
test_df: pd.DataFrame = test.as_dataframe()
train_df: pd.DataFrame = train.as_dataframe()

We'll shuffle the data just for good measure.

In [5]:
train_df = train_df.sample(frac=1.0, random_state=1701)  # shuffle the training data

## Featurize the Molecules with `mordred(community)`
We use `mordred` to calculate a vector of molecular descriptors for each species in this dataset, and then do some re-scaling and imputing to prepare the data for use.

In [6]:
from mordred import Calculator, descriptors
from rdkit.Chem import MolFromSmiles

In [7]:
calc = Calculator(descriptors, ignore_3D=True)

In [8]:
train_features = calc.pandas(map(MolFromSmiles, train_df["smiles"]), nmols=len(train_df)).fill_missing()
test_features = calc.pandas(map(MolFromSmiles, test_df["smiles"]), nmols=len(test_df)).fill_missing()

100%|██████████| 111/111 [00:02<00:00, 51.88it/s]
/home/jackson/neural-pairwise-regression/.venv/lib/python3.12/site-packages/mordred/_base/pandas_module.py:17: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  t[t.applymap(is_missing)] = value
100%|██████████| 24/24 [00:00<00:00, 37.59it/s]
/home/jackson/neural-pairwise-regression/.venv/lib/python3.12/site-packages/mordred/_base/pandas_module.py:17: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  t[t.applymap(is_missing)] = value


In [9]:
train_features

,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,16.958632,13.546673,0,0,28.642859,2.441470,4.882941,28.642859,1.301948,4.021066,...,10.014984,56.167647,297.111341,8.030036,1054,35,114.0,134.0,6.777778,5.000000
1,24.903216,18.124771,0,1,40.614930,2.512441,4.988877,40.614930,1.310159,4.391017,...,10.551402,81.473765,416.232460,7.054787,2779,55,174.0,211.0,9.694444,6.611111
2,19.677670,15.258529,0,0,33.898189,2.409826,4.780314,33.898189,1.355928,4.161928,...,10.066329,73.304213,331.143310,7.884365,1597,37,132.0,155.0,6.027778,5.583333
3,14.417934,12.400650,1,0,24.433289,2.360850,4.721699,24.433289,1.285963,3.853830,...,9.695294,51.990047,254.094294,7.699827,724,28,94.0,108.0,6.916667,4.277778
4,25.093486,20.067761,1,0,41.437036,2.401507,4.803014,41.437036,1.255668,4.400419,...,10.256992,68.743553,452.267508,6.554602,3368,50,164.0,188.0,11.361111,7.472222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,16.401355,13.178323,0,0,27.516148,2.357497,4.672669,27.516148,1.310293,4.000209,...,9.789030,72.959236,282.111676,8.060334,1031,28,108.0,124.0,5.916667,4.638889
107,14.170645,11.707760,0,1,23.935088,2.388155,4.712246,23.935088,1.329727,3.834465,...,9.677277,64.521606,244.132411,7.180365,641,24,94.0,109.0,4.555556,3.972222
108,19.746619,14.936089,0,1,33.362088,2.495481,4.865302,33.362088,1.334484,4.165519,...,10.149449,74.204986,334.142976,7.770767,1610,39,134.0,159.0,6.638889,5.527778
109,24.307097,17.264108,0,1,39.555540,2.472253,4.857191,39.555540,1.318518,4.376647,...,10.412081,85.616131,425.152161,8.021739,2831,46,168.0,199.0,8.312500,6.277778


In [10]:
import lightning
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.callbacks.model_checkpoint import ModelCheckpoint
import torch
import numpy as np

In [11]:
X = torch.tensor(train_features.to_numpy(dtype=np.float32), dtype=torch.float32)
y = torch.tensor(train_df["LOG_RPPB"].to_numpy(dtype=np.float32), dtype=torch.float32)[:, None]  # keep it 2d!
X_test = torch.tensor(test_features.to_numpy(dtype=np.float32), dtype=torch.float32)

In [12]:
val_idx = 12  # use n/110 for validation

In [13]:
from fastprop.data import standard_scale, inverse_standard_scale

In [14]:
X[val_idx:], means, vars = standard_scale(X[val_idx:])
X[:val_idx] = standard_scale(X[:val_idx], means, vars)
X_test = standard_scale(X_test, means, vars)
# sorta-Winsorization
X.clamp_(-3, 3)
X_test.clamp_(-3, 3)

tensor([[-1.5259, -1.6208, -0.1925,  ..., -1.7424, -0.9804, -1.3396],
        [ 0.4879,  0.4359, -0.1925,  ...,  0.4292,  0.1037,  0.6367],
        [ 0.2321,  0.2587, -0.1925,  ...,  0.2722,  0.4036,  0.0473],
        ...,
        [ 0.0987,  0.5012, -0.1925,  ...,  0.0367,  1.1153,  0.1860],
        [ 1.6076,  1.5861, -0.1925,  ...,  1.7635,  1.5107,  1.6191],
        [-1.8798, -1.8199, -0.1925,  ..., -1.5592, -1.7152, -2.0678]])

We could also rescale the targets like this:

```python
y, target_means, target_vars = standard_scale(y)
```

But their natural scale is already pretty close to what we want, so we won't bother (i've tried, and it doesn't significantly impact performance).

## Implementing Pairwise Regression

`nepare` provides a number of convenience classes than handle training, validation, and testing augmentation automatically.

In [15]:
from nepare.nn import NeuralPairwiseRegressor as NPR
from nepare.data import PairwiseAugmentedDataset, PairwiseAnchoredDataset, PairwiseInferenceDataset
from nepare.inference import predict

In [16]:
training_dataset = PairwiseAugmentedDataset(X[val_idx:], y[val_idx:], how='full')
validation_dataset = PairwiseAnchoredDataset(X[val_idx:], y[val_idx:], X[:val_idx], y[:val_idx], how='full')
predict_dataset = PairwiseInferenceDataset(X[val_idx:], y[val_idx:], X_test, how='full')
train_loader = torch.utils.data.DataLoader(training_dataset, batch_size=64, shuffle=True)
val_loader = torch.utils.data.DataLoader(validation_dataset, batch_size=64)
predict_loader = torch.utils.data.DataLoader(predict_dataset, batch_size=64)

These networks can overfit very quickly, so we will use `EarlyStopping` to stop training once we start overfitting and then reset the network to to _just before_ it overfit.

In [17]:
npr = NPR(X.shape[1], 50, 2)
early_stopping = EarlyStopping(monitor="validation/loss", patience=10)
model_checkpoint = ModelCheckpoint(monitor="validation/loss")

In [18]:
trainer = lightning.Trainer(max_epochs=50, log_every_n_steps=1, callbacks=[early_stopping, model_checkpoint])
trainer.fit(npr, train_loader, val_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type       | Params | Mode 
--------------------------------------------
0 | fnn  | Sequential | 163 K  | train
--------------------------------------------
163 K     Trainable params
0         Non-trainable params
163 K     Total params
0.656     Total estimated model params size (MB)
6         Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [19]:
npr = NPR.load_from_checkpoint(model_checkpoint.best_model_path)  # reload best model based on early stopping

In [20]:
y_pred, y_stdev = predict(npr, predict_loader, how="all")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

If we had re-scaled the target, we would have to undo that scaling like this:

```python
y_pred = inverse_standard_scale(torch.tensor(y_pred), target_means, target_vars)
```

## Finally, Results!

In [21]:
results = benchmark.evaluate(y_pred)

In [22]:
results.name = "nepare"
results.description = "Neural Pairwise Regression with Mordred(-community) Molecular Descriptors"
results.github_url = "https://github.com/JacksonBurns/neural-pairwise-regression/blob/main/notebooks/polaris_adme.ipynb"

In [23]:
results

{
  "name": "nepare",
  "description": "Neural Pairwise Regression with Mordred(-community) Molecular Descriptors",
  "tags": [],
  "user_attributes": {},
  "owner": null,
  "polaris_version": "0.11.8",
  "github_url": "https://github.com/JacksonBurns/neural-pairwise-regression/blob/main/notebooks/polaris_adme.ipynb",
  "paper_url": null,
  "contributors": [],
  "results": [
    {
      "test_set": "test",
      "target_label": "LOG_RPPB",
      "scores": {
        "mean_absolute_error": 0.38180363990212846,
        "mean_squared_error": 0.22988774805758586,
        "r2": 0.7412578398962673,
        "pearsonr": 0.8625079254520818,
        "spearmanr": 0.8278260869565216,
        "explained_var": 0.7439059341245777
      }
    }
  ],
  "benchmark_artifact_id": "polaris/adme-fang-rppb-1",
  "benchmark_name": null,
  "benchmark_owner": null,
  "slug": "nepare",
  "artifact_id": null,
  "urn": null,
  "version": 1
}

As of writing, this method lands at third on the leaderboard just barely losing out to a couple 1 _billion_ parameter MPNN-based foundation models.
We've achieved pretty similar performance (without any tuning!) in just a few minutes - pretty good!

This last line is commented out because it will fail (unless you are me) - you can replace the `owner` without your own name to upload your results (and also update the link, name, and description above).

In [25]:
results.upload_to_hub(owner="jacksonburns", access="public")

[11:43:16] Your result has been successfully uploaded to the Hub. View it here:                       ]8;id=671809;file:///home/jackson/neural-pairwise-regression/.venv/lib/python3.12/site-packages/polaris/hub/client.py\client.py]8;;\:]8;id=796352;file:///home/jackson/neural-pairwise-regression/.venv/lib/python3.12/site-packages/polaris/hub/client.py#532\532]8;;\
           https://polarishub.io/benchmarks/polaris/adme-fang-rppb-1/omWNUdZVzkFKp72XGxn4K                         

            Success: Uploading results                                                                ]8;id=249530;file:///home/jackson/neural-pairwise-regression/.venv/lib/python3.12/site-packages/polaris/utils/context.py\context.py]8;;\:]8;id=345356;file:///home/jackson/neural-pairwise-regression/.venv/lib/python3.12/site-packages/polaris/utils/context.py#53\53]8;;\